# Lab 7B: MCP Catalog with Azure API Center 🛰️

Use **Azure API Center** to maintain a catalog of approved MCP servers for your organization.

## Why a Catalog?

| Problem | Solution |
|---------|----------|
| Developers don't know what MCP servers exist | Central registry |
| No visibility into approved tools | Curated list |
| Hard to find endpoint URLs | Discovery API |

## What We'll Do

1. **Deploy API Center** - Create the catalog
2. **Register MCP Servers** - Add approved tools (including NASA space tools! 🚀)
3. **Query the Catalog** - List available MCP servers

---

## Setup

In [ ]:
import os
import json
import hashlib
import requests
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from IPython.display import display, HTML, Markdown
import pandas as pd

load_dotenv("../../.env")

# Configuration
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP", "foundry-mcp-rg")
LOCATION = os.environ.get("LOCATION", "eastus")

# Use deterministic hash (subscription + RG) for cross-user uniqueness
def get_unique_suffix():
    import subprocess
    result = subprocess.run(["az", "account", "show", "--query", "id", "-o", "tsv"], capture_output=True, text=True)
    sub_id = result.stdout.strip() if result.returncode == 0 else "default"
    # Create stable hash from subscription + resource group
    return hashlib.md5(f"{sub_id}-{RESOURCE_GROUP}".encode()).hexdigest()[:6]

UNIQUE_SUFFIX = os.environ.get("UNIQUE_SUFFIX", get_unique_suffix())
API_CENTER_NAME = f"mcp-catalog-{UNIQUE_SUFFIX}"

display(Markdown("## Configuration"))
display(HTML(f"""
<table style="border-collapse: collapse; width: 400px; background: #1e1e2e; color: #cdd6f4;">
<tr><td style="padding: 8px; border: 1px solid #45475a;"><b>Resource Group</b></td><td style="padding: 8px; border: 1px solid #45475a;">{RESOURCE_GROUP}</td></tr>
<tr><td style="padding: 8px; border: 1px solid #45475a;"><b>Location</b></td><td style="padding: 8px; border: 1px solid #45475a;">{LOCATION}</td></tr>
<tr><td style="padding: 8px; border: 1px solid #45475a;"><b>API Center</b></td><td style="padding: 8px; border: 1px solid #45475a;">{API_CENTER_NAME}</td></tr>
</table>
"""))

In [ ]:
# Get Azure credentials and subscription
credential = DefaultAzureCredential()

import subprocess
result = subprocess.run(["az", "account", "show", "-o", "json"], capture_output=True, text=True)
account = json.loads(result.stdout)
SUBSCRIPTION_ID = account["id"]

print(f"✅ Subscription: {account['name']}")

---

## Part 1: Deploy API Center

In [3]:
# Create resource group
!az group create --name {RESOURCE_GROUP} --location {LOCATION} -o none
print(f"✅ Resource group ready")

✅ Resource group ready


In [ ]:
%%bash -s "$RESOURCE_GROUP" "$API_CENTER_NAME" "$LOCATION"

# Deploy API Center using ARM REST API (no extension needed)
RESOURCE_GROUP=$1
API_CENTER_NAME=$2
LOCATION=$3

echo "🚀 Deploying API Center..."

# Check if exists
EXISTS=$(az resource show \
    --resource-group "$RESOURCE_GROUP" \
    --resource-type "Microsoft.ApiCenter/services" \
    --name "$API_CENTER_NAME" \
    --query "name" -o tsv 2>/dev/null || echo "")

if [ -z "$EXISTS" ]; then
    # Create using generic az resource create
    az resource create \
        --resource-group "$RESOURCE_GROUP" \
        --resource-type "Microsoft.ApiCenter/services" \
        --name "$API_CENTER_NAME" \
        --location "$LOCATION" \
        --properties '{}' \
        -o none
    echo "✅ API Center created: $API_CENTER_NAME"
else
    echo "✅ API Center exists: $API_CENTER_NAME"
fi

---

## Part 2: Register MCP Servers

Use the **ARM REST API** to register MCP servers in the catalog.

In [5]:
# Define MCP servers to register - including NASA Space tools! 🚀
MCP_SERVERS = [
    {
        "name": "github-mcp",
        "title": "GitHub MCP Server",
        "description": "Access GitHub repos, issues, and PRs",
        "url": "https://api.githubcopilot.com/mcp",
        "kind": "mcp",
        "category": "Developer Tools",
        "icon": "🐙"
    },
    {
        "name": "nasa-dsn-mcp",
        "title": "NASA Deep Space Network",
        "description": "Query DSN antenna status and communicate with deep space missions",
        "url": "https://api.githubcopilot.com/mcp",
        "kind": "mcp",
        "category": "Space Operations",
        "icon": "📡"
    },
    {
        "name": "nasa-horizons-mcp",
        "title": "NASA JPL Horizons",
        "description": "Get ephemeris data for solar system objects - planets, moons, asteroids",
        "url": "https://api.githubcopilot.com/mcp",
        "kind": "mcp",
        "category": "Space Science",
        "icon": "🪐"
    },
    {
        "name": "iss-tracker-mcp",
        "title": "ISS Tracking & Telemetry",
        "description": "Real-time ISS position, crew info, and live telemetry feeds",
        "url": "https://api.githubcopilot.com/mcp",
        "kind": "mcp",
        "category": "Space Operations",
        "icon": "🛸"
    },
    {
        "name": "nasa-images-mcp",
        "title": "NASA Image & Video Library",
        "description": "Search NASA's media archive - Mars rovers, Hubble, Webb telescope",
        "url": "https://api.githubcopilot.com/mcp",
        "kind": "mcp",
        "category": "Space Science",
        "icon": "🔭"
    },
    {
        "name": "satellite-ops-mcp",
        "title": "Satellite Command & Control",
        "description": "Send commands to Earth observation satellites (authorized users only)",
        "url": "https://api.githubcopilot.com/mcp",
        "kind": "mcp",
        "category": "Space Operations",
        "icon": "🛰️"
    }
]

# Display as a nice HTML table
display(Markdown("## 📋 MCP Servers to Register"))

rows = ""
for i, s in enumerate(MCP_SERVERS):
    bg = "background: #313244;" if i % 2 == 0 else "background: #1e1e2e;"
    rows += f"""<tr style="{bg}">
        <td style="padding: 10px; border: 1px solid #45475a; font-size: 24px; text-align: center;">{s["icon"]}</td>
        <td style="padding: 10px; border: 1px solid #45475a; color: #89b4fa;"><b>{s["title"]}</b></td>
        <td style="padding: 10px; border: 1px solid #45475a; color: #a6adc8;">{s["category"]}</td>
        <td style="padding: 10px; border: 1px solid #45475a; color: #bac2de;">{s["description"][:50]}...</td>
    </tr>"""

display(HTML(f"""
<table style="border-collapse: collapse; width: 100%; background: #1e1e2e;">
<thead><tr style="background: #181825; color: #cdd6f4;">
    <th style="padding: 10px; border: 1px solid #45475a;"></th>
    <th style="padding: 10px; border: 1px solid #45475a;">Name</th>
    <th style="padding: 10px; border: 1px solid #45475a;">Category</th>
    <th style="padding: 10px; border: 1px solid #45475a;">Description</th>
</tr></thead>
<tbody>{rows}</tbody>
</table>
"""))

## 📋 MCP Servers to Register

,Name,Category,Description
🐙,GitHub MCP Server,Developer Tools,"Access GitHub repos, issues, and PRs..."
📡,NASA Deep Space Network,Space Operations,Query DSN antenna status and communicate with deep...
🪐,NASA JPL Horizons,Space Science,Get ephemeris data for solar system objects - plan...
🛸,ISS Tracking & Telemetry,Space Operations,"Real-time ISS position, crew info, and live teleme..."
🔭,NASA Image & Video Library,Space Science,"Search NASA's media archive - Mars rovers, Hubble,..."
🛰️,Satellite Command & Control,Space Operations,Send commands to Earth observation satellites (aut...


In [6]:
# Get access token for ARM API
token = credential.get_token("https://management.azure.com/.default")

ARM_BASE = f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.ApiCenter/services/{API_CENTER_NAME}"
API_VERSION = "2024-03-01"

headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

print(f"✅ Ready to call ARM API")

✅ Ready to call ARM API


In [7]:
# Register each MCP server as an API
display(Markdown("## 🔧 Registering MCP Servers"))

rows = ""
for server in MCP_SERVERS:
    api_url = f"{ARM_BASE}/workspaces/default/apis/{server['name']}?api-version={API_VERSION}"
    
    api_payload = {
        "properties": {
            "title": server["title"],
            "description": server["description"],
            "kind": server["kind"]
        }
    }
    
    response = requests.put(api_url, headers=headers, json=api_payload)
    status = "✅ Registered" if response.status_code in [200, 201] else f"❌ Error {response.status_code}"
    bg_color = "#1e3a29" if response.status_code in [200, 201] else "#3a1e1e"
    text_color = "#a6e3a1" if response.status_code in [200, 201] else "#f38ba8"
    
    rows += f"""<tr style="background: #1e1e2e;">
        <td style="padding: 8px; border: 1px solid #45475a; font-size: 20px;">{server["icon"]}</td>
        <td style="padding: 8px; border: 1px solid #45475a; color: #cdd6f4;">{server["title"]}</td>
        <td style="padding: 8px; border: 1px solid #45475a; background: {bg_color}; color: {text_color};">{status}</td>
    </tr>"""

display(HTML(f"""
<table style="border-collapse: collapse; width: 100%; background: #1e1e2e;">
<thead><tr style="background: #181825; color: #cdd6f4;">
    <th style="padding: 8px; border: 1px solid #45475a;"></th>
    <th style="padding: 8px; border: 1px solid #45475a;">Server</th>
    <th style="padding: 8px; border: 1px solid #45475a;">Status</th>
</tr></thead>
<tbody>{rows}</tbody>
</table>
"""))
print("\n🎉 Registration complete!")

## 🔧 Registering MCP Servers

,Server,Status
🐙,GitHub MCP Server,✅ Registered
📡,NASA Deep Space Network,✅ Registered
🪐,NASA JPL Horizons,✅ Registered
🛸,ISS Tracking & Telemetry,✅ Registered
🔭,NASA Image & Video Library,✅ Registered
🛰️,Satellite Command & Control,✅ Registered



🎉 Registration complete!


In [8]:
# Add versions and deployments with endpoint URLs
display(Markdown("## 🔗 Configuring Endpoints"))

for server in MCP_SERVERS:
    # Create version
    version_url = f"{ARM_BASE}/workspaces/default/apis/{server['name']}/versions/v1?api-version={API_VERSION}"
    version_payload = {"properties": {"title": "v1", "lifecycleStage": "production"}}
    requests.put(version_url, headers=headers, json=version_payload)
    
    # Create definition with endpoint URL
    def_url = f"{ARM_BASE}/workspaces/default/apis/{server['name']}/versions/v1/definitions/default?api-version={API_VERSION}"
    def_payload = {"properties": {"title": "MCP Endpoint", "description": f"Endpoint: {server['url']}"}}
    requests.put(def_url, headers=headers, json=def_payload)

# Show endpoint summary
rows = ""
for i, s in enumerate(MCP_SERVERS):
    bg = "background: #313244;" if i % 2 == 0 else "background: #1e1e2e;"
    rows += f"""<tr style="{bg}">
        <td style="padding: 8px; border: 1px solid #45475a;">{s["icon"]}</td>
        <td style="padding: 8px; border: 1px solid #45475a; color: #cdd6f4;">{s["title"]}</td>
        <td style="padding: 8px; border: 1px solid #45475a; font-family: monospace; font-size: 12px; color: #94e2d5;">{s["url"]}</td>
    </tr>"""

display(HTML(f"""
<table style="border-collapse: collapse; width: 100%; background: #1e1e2e;">
<thead><tr style="background: #1e3a5f; color: #89b4fa;">
    <th style="padding: 8px; border: 1px solid #45475a;"></th>
    <th style="padding: 8px; border: 1px solid #45475a;">Server</th>
    <th style="padding: 8px; border: 1px solid #45475a;">Endpoint URL</th>
</tr></thead>
<tbody>{rows}</tbody>
</table>
<div style="margin-top: 15px; padding: 10px; background: #1e3a29; border-radius: 5px; color: #a6e3a1; border: 1px solid #45475a;">
    ✅ All endpoints configured!
</div>
"""))

## 🔗 Configuring Endpoints

,Server,Endpoint URL
🐙,GitHub MCP Server,https://api.githubcopilot.com/mcp
📡,NASA Deep Space Network,https://api.githubcopilot.com/mcp
🪐,NASA JPL Horizons,https://api.githubcopilot.com/mcp
🛸,ISS Tracking & Telemetry,https://api.githubcopilot.com/mcp
🔭,NASA Image & Video Library,https://api.githubcopilot.com/mcp
🛰️,Satellite Command & Control,https://api.githubcopilot.com/mcp


---

## Part 3: Query the Catalog

List registered MCP servers using the **ARM API**.

In [9]:
# List all registered MCP servers via ARM API
list_url = f"{ARM_BASE}/workspaces/default/apis?api-version={API_VERSION}"
response = requests.get(list_url, headers=headers)

display(Markdown("## 🛰️ MCP Server Catalog"))

if response.status_code == 200:
    apis = response.json().get("value", [])
    
    # Group by category
    categories = {}
    for api in apis:
        props = api.get("properties", {})
        name = api.get("name", "")
        server = next((s for s in MCP_SERVERS if s["name"] == name), {})
        cat = server.get("category", "General")
        if cat not in categories:
            categories[cat] = []
        categories[cat].append({
            "icon": server.get("icon", "📦"),
            "title": props.get("title", name),
            "kind": props.get("kind", "api"),
            "description": props.get("description", "")
        })
    
    # Display by category
    html = ""
    for cat, servers in categories.items():
        html += f"""<h4 style="margin-top: 20px; border-bottom: 2px solid #89b4fa; padding-bottom: 5px; color: #cdd6f4;">{cat}</h4>"""
        for s in servers:
            html += f"""
            <div style="display: flex; align-items: center; padding: 10px; margin: 5px 0; background: #313244; border-radius: 8px; border-left: 4px solid #89b4fa;">
                <span style="font-size: 28px; margin-right: 15px;">{s["icon"]}</span>
                <div>
                    <strong style="color: #cdd6f4;">{s["title"]}</strong>
                    <span style="background: #45475a; color: #a6adc8; padding: 2px 8px; border-radius: 4px; font-size: 11px; margin-left: 10px;">{s["kind"].upper()}</span>
                    <br><span style="color: #a6adc8; font-size: 13px;">{s["description"][:70]}...</span>
                </div>
            </div>"""
    
    display(HTML(html))
    
    # Summary card
    display(HTML(f"""
    <div style="margin-top: 25px; padding: 20px; background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); 
                border-radius: 12px; color: white; text-align: center;">
        <h2 style="margin: 0;">🚀 {len(apis)} MCP Servers Available</h2>
        <p style="margin: 8px 0 0 0; opacity: 0.8;">Ready for AI agent integration</p>
    </div>
    """))
else:
    print(f"❌ Error {response.status_code}: {response.text[:200]}")

## 🛰️ MCP Server Catalog

In [10]:
# Export as MCP config (for use with agents)
display(Markdown("## 📄 Export Configuration for Agents"))

mcp_config = {"mcpServers": {}}

for api in apis:
    name = api.get("name", "")
    props = api.get("properties", {})
    server_data = next((s for s in MCP_SERVERS if s["name"] == name), None)
    
    mcp_config["mcpServers"][name] = {
        "title": props.get("title", name),
        "url": server_data["url"] if server_data else f"https://{name}.local",
        "category": server_data.get("category", "General") if server_data else "General"
    }

# Pretty display
display(HTML("""
<details open>
<summary style="cursor: pointer; font-weight: bold; padding: 10px; background: #313244; border-radius: 5px; color: #cdd6f4; border: 1px solid #45475a;">
    📋 approved-mcp-servers.json
</summary>
<pre style="background: #11111b; color: #cdd6f4; padding: 15px; border-radius: 5px; overflow-x: auto; border: 1px solid #45475a;">
""" + json.dumps(mcp_config, indent=2) + """
</pre>
</details>
"""))

# Save to file
with open("approved-mcp-servers.json", "w") as f:
    json.dump(mcp_config, f, indent=2)

display(HTML("""
<div style="margin-top: 15px; padding: 10px; background: #1e3a29; border-radius: 5px; color: #a6e3a1; border: 1px solid #45475a;">
    ✅ Saved to <code style="background: #45475a; padding: 2px 6px; border-radius: 3px;">approved-mcp-servers.json</code>
</div>
"""))

## 📄 Export Configuration for Agents

---

## Summary 🎉

You've created an **MCP Server Catalog** using Azure API Center!

### What's in Your Catalog

| Category | Servers |
|----------|---------|
| 🐙 Developer Tools | GitHub MCP |
| 📡 Space Operations | NASA DSN, ISS Tracker, Satellite C2 |
| 🔭 Space Science | JPL Horizons, NASA Images |

### How Developers Use This

1. **Query the catalog** → Find approved MCP servers
2. **Get endpoint URLs** → From the registry
3. **Configure agents** → Use only approved servers

### Next Steps

- Set up **API Center Portal** for self-service discovery
- Add **access policies** for sensitive tools (like Satellite C2!)
- Integrate with **CI/CD** for automatic registration

### Portal Link

In [ ]:
print(f"🔗 View in Azure Portal:")
print(f"   https://portal.azure.com/#@/resource/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.ApiCenter/services/{API_CENTER_NAME}/overview")

---

## Cleanup (Optional)

In [12]:
# Uncomment to delete
# !az resource delete --resource-group {RESOURCE_GROUP} --resource-type "Microsoft.ApiCenter/services" --name {API_CENTER_NAME}
# print("🗑️ Deleted")